In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
import os

# Define the function
def train_and_serve_model(data_path, model_path="model.h5"):
    """
    Train a neural network on the NUSW-NB15 dataset and save the model for serving.

    Args:
        data_path (str): Path to the CSV dataset.
        model_path (str): Path to save the trained model (default: "model.h5").
    Returns:
        str: Message indicating training completion and model saving.
    """
    # Load dataset
    data = pd.read_csv(data_path)
    
    # Encode categorical features
    categorical_cols = data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        data[col] = LabelEncoder().fit_transform(data[col])
    
    # Split into features (X) and target (y)
    X = data.drop(columns=['class'], axis=1)  # Assuming 'class' is the target
    y = data['class']
    
    # Normalize features
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    
    # Encode target labels
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded)
    
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)
    
    # Build the neural network
    model = Sequential([
        Dense(64, input_dim=X_train.shape[1], activation='relu'),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(y_categorical.shape[1], activation='softmax')  # Output layer
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model
    model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32, verbose=1)
    
    # Save the trained model
    model.save(model_path)
    
    # Save the scaler and label encoder for consistent preprocessing
    np.save("scaler.npy", scaler)
    np.save("label_encoder.npy", label_encoder.classes_)
    
    return f"Model trained and saved to {model_path}"

def predict(input_data, model_path="model.h5"):
    """
    Predict the class of new input data using the trained model.

    Args:
        input_data (list): List of input features (single instance).
        model_path (str): Path to the trained model (default: "model.h5").
    Returns:
        str: Predicted class label.
    """
    # Load the trained model
    model = load_model(model_path)
    
    # Load the scaler and label encoder
    scaler = MinMaxScaler()
    scaler = scaler.fit(np.load("scaler.npy", allow_pickle=True))
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load("label_encoder.npy", allow_pickle=True)
    
    # Preprocess input data
    input_data = scaler.transform([input_data])
    
    # Predict the class
    predictions = model.predict(input_data)
    predicted_class = label_encoder.inverse_transform([np.argmax(predictions)])
    
    return predicted_class[0]
